In [12]:
from glob import glob
import os

datasets = [foo.split('/')[1] for foo in glob('../*/labels.json')]

if len(datasets) == 0:
    raise FileNotFoundError("no datasets found")

In [42]:
import random

# choose one random utternace id
_, dataset, __, ___, filename = random.choice(glob('../*/raw-videos/train/*')).split('/')
uttid = filename.split('.')[0]

dataset, uttid

('IEMOCAP', 'Ses04M_impro05_F035')

In [45]:
from collections import namedtuple
import json

def get_stuff(dataset, uttid):
    vid_path = os.path.join('../', dataset, 'train', uttid +'.avi')
    if not os.path.isfile(vid_path):
        vid_path = vid_path.replace('.avi', '.mp4')
    if not os.path.isfile(vid_path):
        vid_path = None
        
    audio_path = os.path.join('../', dataset, 'train', uttid + '.wav')

    if not os.path.isfile(audio_path):
        audio_path = audio_path.replace('.wav', '.mp3')
    if not os.path.isfile(audio_path):
        audio_path = None
        
    text_path = os.path.join('../', dataset, 'train', uttid + '.json')
    print(text_path)
    if not os.path.isfile(text_path):
        text_path = None
        
    label_path = os.path.join('../', dataset, 'labels.json')

    with open(label_path, 'r') as stream:
        labels = json.load(stream)
        
    label = labels['train'][uttid]
    
    return vid_path, audio_path, text_path, label

In [46]:
get_stuff(dataset, uttid)

../IEMOCAP/train/Ses04M_impro05_F035.json


(None, None, None, 'frustration')

In [33]:
stuff = namedtuple('stuff', [
    'bbox', 'landmark', 'det_score', 'embedding', 'gender', 'age',
    'embedding_norm', 'normed_embedding'
])

Face.__new__.__defaults__ = (None, ) * len(Face._fields)

face = Face(bbox=bbox,
            landmark=landmark,
            det_score=det_score,
            embedding=embedding,
            gender=gender,
            age=age,
            normed_embedding=normed_embedding,
            embedding_norm=embedding_norm)

Signature:
namedtuple(
    typename,
    field_names,
    *,
    rename=False,
    defaults=None,
    module=None,
)
Docstring:
Returns a new subclass of tuple with named fields.

>>> Point = namedtuple('Point', ['x', 'y'])
>>> Point.__doc__                   # docstring for the new class
'Point(x, y)'
>>> p = Point(11, y=22)             # instantiate with positional args or keywords
>>> p[0] + p[1]                     # indexable like a plain tuple
33
>>> x, y = p                        # unpack like a regular tuple
>>> x, y
(11, 22)
>>> p.x + p.y                       # fields also accessible by name
33
>>> d = p._asdict()                 # convert to a dictionary
>>> d['x']
11
>>> Point(**d)                      # convert from a dictionary
Point(x=11, y=22)
>>> p._replace(x=100)               # _replace() is like str.replace() but targets named fields
Point(x=100, y=22)
File:      /usr/lib/python3.7/collections/__init__.py
Type:      function
